In [1]:
import os
import sys 
import numpy as np 
import tensorflow as tf
import operator

In [5]:
def load_model(session, save_path):
    """ Loads a saved TF model from a file.Returns:The inputs placehoder and the prediction operation. """
    print("Loading model from file '%s'..." % (save_path))
    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        print("ERROR: Expected .meta file '%s', but could not find it." % \
        (meta_file))
        sys.exit(1) 
    saver = tf.train.import_meta_graph(meta_file)
    save_path = os.path.join("./", save_path)
    saver.restore(session, save_path)  
    return extract_validation_handles(session)
    

def load_validation_data(rootpath, model=1):
    global batch_size 
    """ Loads the validation data,  Returns: A tuple of the loaded validation data and validation labels. """
    print("Loading validation data...") 
    if model ==1:
        testX  =np.load(os.path.join(rootpath, 'testX_pred.npy') ) 
        testY  =np.load(os.path.join(rootpath, 'testY_pred.npy'))  
    elif model==2:
        testX  =np.load(os.path.join(rootpath, 'testX_no.npy') ) 
        testY  =np.load(os.path.join(rootpath, 'testY_no.npy'))  
    if testX.shape[1]!= 214:
        testX = np.transpose(testX,[0,2,1])
        testX = np.expand_dims(testX,3) 
    batch_size =np.shape(testX)[0]    
    return (testX,testY )  

def validate_model(session, val_data, x, y,  phase_train,prob): 
    """ Validates the model stored in a session.Returns:The overall validation accuracy for the model. """
    global batch_size
    print("Validating model...") 
    predict_op = tf.argmax(prob, 1)
    correct = tf.equal(predict_op,tf.argmax(y, 1))
    acc= tf.reduce_mean(tf.cast(correct, tf.float32))  
    val_x, val_y  = val_data
    test_correct = session.run(correct,feed_dict ={x: val_x,  y:val_y, phase_train: True})
    test_acc = session.run(acc,feed_dict ={x: val_x, y:val_y, phase_train: True})
    return test_acc, test_correct 
 
def extract_validation_handles(session):
    """ Extracts the input and predict_op handles that we use for validation.
    Args:
      session: The session with the loaded graph.
    Returns:
      The inputs placeholder, mask placeholder, and the prediction operation. """ 
    valid_nodes = tf.get_collection_ref("validation_nodes") 
    '''if len(valid_nodes) != 4:
        print("ERROR: Expected 4 items in validation_nodes, got %d." %(len(valid_nodes)))
        sys.exit(1) ''' 
    x = valid_nodes[0] 
    y = valid_nodes[1] 
    phase_train =valid_nodes[2]
    prob =  valid_nodes[3]
    global batch_size  
    return (x,y,  phase_train,prob)

def each_perform(correct_results,eval_labels ):
    label_y=np.argmax(eval_labels,1)
    acc = []
    
    zero = np.where(label_y==0)
    correctl=[correct_results[i] for i in zero]
    accuracy_zero=1-np.mean(correctl)
    acc.append(accuracy_zero)
    
    one = np.where(label_y==1)
    correctg=[correct_results[i] for i in one]
    accuracy_one=1-np.mean(correctg)
    acc.append(accuracy_one)
    
    two = np.where(label_y==2)
    correctt=[correct_results[i] for i in two]
    accuracy_two=1-np.mean(correctt) 
    acc.append(accuracy_two)  
    return acc, correctl,correctg,correctt 
def reset():
    tf.reset_default_graph()

In [11]:
batch_size =50  
model =1# model = 1: SP process; model = 2: NS process 
root = '/data/test/time_series_data'
save_path = '/codes/saved_models/ModelC/modelC_fully' 
testname = 'one_sec'  # choose from{  total,one_sec,two_sec, half_sec, one_half_sec } 
rootpath = os.path.join(root, testname) 
with tf.Session() as session:   
    x, y,phase_train,prob  = load_model(session, save_path)
    val_data = load_validation_data(rootpath, model = model) 
    accuracy ,correct_results= validate_model(session, val_data, x,y, phase_train,prob) 
    print ("Overall validation accuracy is %f \n" %(accuracy))
    acc,correct_l,correct_g,correct_t = each_perform(correct_results,val_data[1] )  
    session.close()     
id_True =  np.nonzero(correct_results==True) 
overall_acc = (len(id_True[0])/len(correct_results))    
print ('LT:', 100*np.mean( correct_l))
print ('GT:', 100*np.mean( correct_g))
print ('TP:', 100*np.mean( correct_t)) 
print (  100*overall_acc)  

Loading model from file 'C:/07_research/14_successive_events/codes/saved_models/ModelC/modelC_fully'...
INFO:tensorflow:Restoring parameters from C:/07_research/14_successive_events/codes/saved_models/ModelC/modelC_fully
Loading validation data...
Validating model...
Overall validation accuracy is 0.715210 

LT: 72.16494845360825
GT: 63.366336633663366
TP: 78.37837837837837
71.52103559870551
